In [1]:
import pymongo
from pymongo import MongoClient
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import seaborn as sns
from bson.son import SON
from bson.code import Code
import json
import pandas as pd
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3
# These are packages need for natural language processing:
import nltk
from __future__ import division, unicode_literals 
from textblob import TextBlob as tb
import math
from datetime import datetime
from dateutil import parser

In [2]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import amazon_data_sort 
from amazon_data_sort import get_b_product, get_reviews
import doc_summarization as ds

In [7]:
#For Topic Modelling LDA:
import lda
import scipy
import gensim

from gensim import corpora, models, similarities
from gensim.models import hdpmodel, ldamodel
from itertools import izip

In [8]:
client = MongoClient()
mbf = client.dsbc.mbf
product_info = client.dsbc.product_info
name = product_info.find({}, {"_id":0,"asin":1,"title":1,"price":1})
beauty_list  = {}
for id_num in name:
    try:
        beauty_list[id_num["asin"]]=[id_num["title"],id_num["price"]]
    except:
        pass
#Creating database for reviews beauty file
rbf = client.dsbc.rbf

###[u'Olay Professional Pro-X Hydra Firming Cream Anti Aging 1.7 Oz', 15.38]

In [ ]:
text = '-'.join(ds.summarize(get_b_product(beauty_list,3)['Reviews'])['mean_score_summary'])
wordcloud1 = WordCloud(font_path='/Library/Fonts/AppleSDGothicNeo-ExtraBold.otf',
                      width=800, height=800,
                      stopwords=STOPWORDS.update(["buy","use", "product", "skin", "will", "cream", "year",
                                                  "face", "good","using", "products", "really", "see", "olayX","olay", 
                                                  "price", "make", "pro", "look", "day", "years", "feel", "line",
                                                  "used", "use", "x", "much"])).generate(text)
print text
plt.imshow(wordcloud1)
plt.axis("off")

In [ ]:
wordcloud1 = WordCloud(font_path='/Library/Fonts/AppleSDGothicNeo-ExtraBold.otf',
                      width=800, height=800,
                      stopwords=STOPWORDS.update(["buy","use", "product", "skin", "will", "cream", "year",
                                                  "face", "good","using", "products", "really", "see", "olayX","olay", 
                                                  "price", "make", "pro", "look", "day", "years", "feel", "line",
                                                  "used", "use", "x", "much"])).generate(get_b_product(beauty_list,3)['Reviews'])
print text
plt.imshow(wordcloud1)
plt.axis("off")

In [ ]:
get_reviews(beauty_list,43)

In [9]:
def skin_type_counter(list_of_text):
    dry_skin = 0
    oily_skin = 0
    combination_skin = 0
    sensitivity = 0
    total_reviews = len(list_of_text)
    for review_text in list_of_text:
        new_text = review_text[0].lower()
        dry_skin += new_text.count("dry skin")
        oily_skin += new_text.count("oily skin")
        combination_skin += new_text.count("combination skin")
        sensitivity += new_text.count("sensitive")*tb(new_text).sentiment.polarity
    return [dry_skin, oily_skin, combination_skin, sensitivity/(total_reviews+1)]
    

In [11]:
a = get_reviews(beauty_list,42)['Reviews']
# for i in a:
#     print i
#     print skin_type_counter([i])
#     print ""
#     print ""

In [15]:
words = []
sentences  = nltk.word_tokenize(a[0][0])

In [28]:
stopwords = nltk.corpus.stopwords.words("english")\
            + ['usps','shipping','$','#', 'very', 'amazon','every',
                '&','order','seller','packaging','manufacture', 'ever',
                'package','cute','delivery', '.',',',"return", "might"
                '?','...','day','week', 'weeks']


In [29]:
for sentence in sentences:
    tokens = nltk.word_tokenize(sentence)
    text = [word for word in tokens if word not in stopwords]
    tagged_text = nltk.pos_tag(text)
    for word, tag in tagged_text:
        words.append({"word": word, "pos": tag})

In [32]:
from nltk.stem.wordnet import WordNetLemmatizer

In [35]:
words

[{u'pos': 'NN', u'word': u'worth'},
 {u'pos': 'NN', u'word': u'price'},
 {u'pos': 'NN', u'word': u'eradicate'},
 {u'pos': 'NN', u'word': u'keratosis'},
 {u'pos': 'NNS', u'word': u'pilaris'},
 {u'pos': 'RB', u'word': u"n't"},
 {u'pos': 'NN', u'word': u'aware'},
 {u'pos': 'RB', u'word': u'little'},
 {u'pos': 'NNS', u'word': u'bumps'},
 {u'pos': 'NN', u'word': u'get'},
 {u'pos': 'NN', u'word': u'skin'},
 {u'pos': 'RB', u'word': u'often'},
 {u'pos': 'VBD', u'word': u'referred'},
 {u'pos': '``', u'word': u'``'},
 {u'pos': 'NN', u'word': u'chicken'},
 {u'pos': 'NN', u'word': u'skin'},
 {u'pos': "''", u'word': u"''"},
 {u'pos': 'NN', u'word': u'way'},
 {u'pos': 'NNS', u'word': u'looks'},
 {u'pos': 'NNS', u'word': u'feels'},
 {u'pos': 'NN', u'word': u'condition'},
 {u'pos': 'JJ', u'word': u'vary'},
 {u'pos': 'NN', u'word': u'severity'},
 {u'pos': 'NNS', u'word': u'bumps'},
 {u'pos': 'VBD', u'word': u'red'},
 {u'pos': 'IN', u'word': u'like'},
 {u'pos': 'NN', u'word': u'acne'},
 {u'pos': 'IN', u

In [41]:
nouns = []
lem = WordNetLemmatizer()
noun_only = [noun for noun in words if noun["pos"] in ["NN", "NNS"]]
for noun_word in noun_only:
    nouns.append(lem.lemmatize(noun_word["word"]))
    

In [44]:
import logging
import gensim
from gensim import corpora
from gensim.corpora import BleiCorpus